# Workshop 2 - Apply
    
In this notebook you will solve a 2-element frame at the end of the notebook.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

## Two-element frame

<figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/twoelemframe.png" WIDTH=300 ALIGN="center">
</figure>

With:
- $EI = 1500$
- $EA = 1000$
- $q = 9$
- $L = 5$
- $\bar\varphi = 0.15$

The final example of this notebook is the two-element frame above. Here you should make use of all the new code you implemented:
    
- Set up the problem and compute a solution for `u_free`. Remember to consider the prescribed horizontal displacement $\bar{u}$ at the right end of the structure.
- Compute and plot bending moment lines for both elements (in the local and global coordinate systems)
- Compute reactions at both supports

In [33]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import matrixmethod as mm

# Clear any previous data
mm.Node.clear()
mm.Element.clear()

# -------------------------------------------------------------------
# Problem parameters (from the exercise)
# -------------------------------------------------------------------
EI = 1500.0
EA = 1000.0
q = 9.0
L = 5.0
prescribed_u = 0.15   # prescribed horizontal displacement at node C

# -------------------------------------------------------------------
# Create nodes
# -------------------------------------------------------------------
# Node A (left bottom, fixed)
nodeA = mm.Node(0.0, 0.0)
# Node B (right bottom, pinned: u and w fixed, rotation free)
nodeB = mm.Node(L, 0.0)
# Node C (top right, free, but with prescribed horizontal displacement)
nodeC = mm.Node(L, L)

# -------------------------------------------------------------------
# Create elements and set section properties
# -------------------------------------------------------------------
# Element 1: horizontal beam from A to B
elem1 = mm.Element(nodeA, nodeB)
elem1.set_section({'EA': EA, 'EI': EI})

# Element 2: vertical column from B to C
elem2 = mm.Element(nodeB, nodeC)
elem2.set_section({'EA': EA, 'EI': EI})

# -------------------------------------------------------------------
# Apply distributed loads
# -------------------------------------------------------------------
# On horizontal element: q downward (negative z-direction)
elem1.add_distributed_load([0.0, -q])          # local: axial=0, transverse=-q

# On vertical element: 2q to the right (positive x-direction)
# For a vertical element with angle 90°, local transverse = -global x
elem2.add_distributed_load([0.0, -2.0*q])      # local transverse = -2q

# -------------------------------------------------------------------
# Boundary conditions and prescribed displacement
# -------------------------------------------------------------------
con = mm.Constrainer()

# Node A: fully fixed (all DOFs zero)
con.fix_dof(nodeA, 0, 0.0)
con.fix_dof(nodeA, 1, 0.0)
con.fix_dof(nodeA, 2, 0.0)

# Node B: pinned – fix horizontal and vertical, leave rotation free
con.fix_dof(nodeB, 0, 0.0)   # u_B = 0
con.fix_dof(nodeB, 1, 0.0)   # w_B = 0
# rotation (dof 2) is free, so no constraint

# Node C: prescribe horizontal displacement, others free
con.fix_dof(nodeC, 0, prescribed_u)   # u_C = 0.15
# w_C and rotation are free

# -------------------------------------------------------------------
# Assemble global stiffness matrix and load vector
# -------------------------------------------------------------------
n_dofs = mm.Node.ndof
K = np.zeros((n_dofs, n_dofs))
F = np.zeros(n_dofs)

# Add contributions from both elements
for elem in [elem1, elem2]:
    dofs = elem.global_dofs()
    k_elem = elem.stiffness()
    for i, dof_i in enumerate(dofs):
        for j, dof_j in enumerate(dofs):
            K[dof_i, dof_j] += k_elem[i, j]

# Nodal loads (already added via add_distributed_load, but we also need to
# include any point loads – here none)
F[nodeA.dofs] += nodeA.p
F[nodeB.dofs] += nodeB.p
F[nodeC.dofs] += nodeC.p

# -------------------------------------------------------------------
# Apply constraints and solve for free displacements
# -------------------------------------------------------------------
Kff, Ff = con.constrain(K, F)
u_free = np.linalg.solve(Kff, Ff)
u_full = con.full_disp(u_free)

print("Free displacements (u_free):")
print(u_free)
print("\nExpected (from hint):")
print("[-0.09274451, -0.13310939, 0.51159348, -0.01644455]")
print("\nFull displacement vector (all DOFs):")
print(u_full)

# -------------------------------------------------------------------
# Compute support reactions
# -------------------------------------------------------------------
reactions = con.support_reactions(K, u_free, F)
print("\nReactions at constrained DOFs (order of cons_dofs):")
print(reactions)
print("Expected (from hint):")
print("[27.35024439, -63.82451092, 17.64975561, -71.17548908, -36.75489076]")

# (Optional) Print the list of constrained DOFs to interpret reactions
print("\nConstrained DOFs (in order):", con.cons_dofs)

# -------------------------------------------------------------------
# Plotting
# -------------------------------------------------------------------
# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# List of elements for looping
elems = [elem1, elem2]

for i, elem in enumerate(elems):
    u_elem = u_full[elem.global_dofs()]

    # Bending moment diagram in local coordinates
    plt.sca(axes[0, 0])
    elem.plot_moment_diagram(u_elem, num_points=50, global_c=False)
    if i == 0:
        axes[0, 0].set_title('Local moment diagram')

    # Bending moment diagram in global coordinates
    plt.sca(axes[0, 1])
    elem.plot_moment_diagram(u_elem, num_points=50, global_c=True)
    if i == 0:
        axes[0, 1].set_title('Global moment diagram')

    # Displaced shape in local coordinates
    plt.sca(axes[1, 0])
    elem.plot_displaced(u_elem, num_points=50, global_c=False, scale=20)
    if i == 0:
        axes[1, 0].set_title('Local displaced shape')

    # Displaced shape in global coordinates
    plt.sca(axes[1, 1])
    elem.plot_displaced(u_elem, num_points=50, global_c=True, scale=20)
    if i == 0:
        axes[1, 1].set_title('Global displaced shape')

plt.tight_layout()
plt.show()

NameError: name 'global_element_load' is not defined

In [ ]:
for elem in elems:
    u_elem = con.full_disp(#YOUR CODE HERE)[#YOUR CODE HERE.global_dofs()]
    elem.plot_displaced #YOUR CODE HERE

SyntaxError: incomplete input (1892221003.py, line 3)

For the given parameter values, if your implementation is fully correct, you should get the following nodal displacements and support reactions:
$$
\mathbf{u}_\mathrm{free} = \left[-0.09274451, -0.13310939,  0.51159348, -0.01644455\right]
$$

$$
\mathbf{f}_\mathrm{cons} = \left[27.35024439, -63.82451092,  17.64975561, -71.17548908, -36.75489076\right]
$$

You should also get the following moment lines for the two elements:

- in local coordinate system:
![](https://raw.githubusercontent.com/ibcmrocha/public/main/moments_local.svg)

- in global coordinate system:
![](https://raw.githubusercontent.com/ibcmrocha/public/main/moments_global.svg)

And the following displacements:
- in local coordinate system:
![](https://raw.githubusercontent.com/ibcmrocha/public/main/displacements_local.svg)

- in global coordinate system:
![](https://raw.githubusercontent.com/ibcmrocha/public/main/displacements_global.svg)